# 2024-2 URP assignment

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import re
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time

In [2]:
# 데이터프레임 선언
Q_df = pd.DataFrame(columns = ['문서 번호', '제목',  '질문',  "위치 고유번호" , '조회수', '질문 날짜'])
A_df = pd.DataFrame(columns = ['문서 번호', '답변 ID',  '답변', '답변 날짜'])
loc_dict = pd.read_csv("naver_kin_regions_df_l2.csv", index_col = "지역 코드")

In [3]:
local_lists = loc_dict.index.tolist()
local_lists

[120115,
 120116,
 120117,
 120118,
 120119,
 120120,
 120121,
 120122,
 120123,
 120124,
 120125,
 120126,
 120127,
 120128,
 120129,
 120130,
 120131,
 120132,
 120133,
 120134,
 120135,
 120136,
 120137,
 120138,
 120139,
 120203,
 120204,
 120205,
 120206,
 120207,
 120208,
 120209,
 120210,
 120211,
 120212,
 120213,
 120214,
 120215,
 120216,
 120217,
 120218,
 120306,
 120307,
 120308,
 120309,
 120310,
 120311,
 120312,
 120313,
 120404,
 120405,
 120406,
 120407,
 120408,
 120409,
 120410,
 120411,
 120412,
 120413,
 120504,
 120505,
 120506,
 120507,
 120508,
 120603,
 120604,
 120605,
 120606,
 120607,
 120701,
 120702,
 120703,
 120704,
 120705,
 120802,
 120803,
 120804,
 120805,
 120806,
 120807,
 120808,
 120809,
 120810,
 120811,
 120812,
 120813,
 120814,
 120815,
 120816,
 120817,
 120818,
 120819,
 120905,
 120906,
 120907,
 120908,
 120909,
 120910,
 120911,
 120912,
 120913,
 120914,
 120915,
 120916,
 120917,
 120918,
 120919,
 120920,
 120921,
 120922,
 120923,
 

# URL에서 dirID 값을 가져오는 함수

In [4]:
def get_dirid(url):
    match = re.search(r'dirId=(\d+)', url)
    dirid_value = match.group(1)
    if len(dirid_value) >= 6 :
        dirid_value = dirid_value[:6]
    return dirid_value

# URL에서 docID 값을 가져오는 함수

In [5]:
def get_docid(url):
    match = re.search(r'docId=(\d+)', url)
    docid_value = match.group(1)
    return docid_value

# 반복되는 패턴을 통해서 docID를 통해 위치 반환하는 함수(추후 매핑 테이블을 이용할 예정)

In [6]:
# dirID -> loc
def get_loc(id=""):
    url = 'https://kin.naver.com/qna/list.naver?dirId=12'+id

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')

    location = soup.find(attrs={'class': 'location'}).get_text(strip=True)
    return location

In [7]:
def find_pattern(text):
    # 문자열 길이의 절반까지 탐색(더 긴 패턴은 두 번 반복될 수 없으므로)
    for length in range(1, len(text)):
        # 길이 `length`만큼 패턴을 잘라서 비교
        pattern = text[:length]
        # 패턴이 두 번 연속 반복되는지 확인
        if text.startswith(pattern * 2):
            return pattern

In [8]:
# 지역 & 플레이스 기본 정보 문자열 제거를 위한 변수 지정
url2 = 'https://kin.naver.com/qna/list.naver?dirId=12'
headers2 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response2 = requests.get(url2, headers=headers2)
soup2 = BeautifulSoup(response2.text, 'html.parser')
useless_information = soup2.find(attrs={'class': 'location'}).get_text(strip=True)

def return_location(search):
    a = get_loc(search[:-2]+"01").replace(get_loc(search[:-2]),"")
    a_elim = get_loc(search[:-len(search)]+search[-len(search):-2]).replace(get_loc(search[:-len(search)]),"")
    first_loc = find_pattern(a)
    a = a[len(first_loc):]
    b = get_loc(search).replace(a,"")
    c = b.replace(useless_information, "")
    return c.replace(a_elim,"")

def return_full_location(search):
    full_location = ""
    n = 2
    while n <= len(search):
        full_location += return_location(search[:n])+ " "
        n += 2
    return full_location

# 최종적으로 질문과 답변 정보 받아오는 함수

In [9]:
def collect_information(url, Q_df, A_df):
    # 페이지 요청
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    # BeautifulSoup으로 페이지 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 질문 제목 추출
    title = soup.find(attrs={'class': 'endTitleSection'}).get_text(strip=True)
    title = title.replace(",","")
    Q_detail= soup.find(attrs={'class': 'questionDetail'}).get_text(strip=True,separator=' ')
    Q_detail = Q_detail.replace(",","")
    Q_info = soup.find_all(class_='infoItem')
    Q_view = Q_info[0].get_text()[3:]
    Q_date = Q_info[1].get_text()[3:]
    if Q_date.endswith(" 전"):
        Q_date = datetime.today().strftime('%Y.%m.%d')
    if Q_date.startswith("작성일"):
        Q_date = Q_date.replace("작성일", "")
    ans = []
    A_dates = []
    try:
        answers = soup.find_all(class_='se-main-container')
        ans_date = soup.find_all(class_='answerDate')
        for answer in answers:
            ans.append(answer.get_text(strip=True).replace('\u200b', ''))
        for ans_dates in ans_date:
            A_dates.append(ans_dates.get_text(strip=True))
    except:
        answer_error = "아직 답변이 없습니다"
    
    ID = get_dirid(url)
    doc_ID = get_docid(url)
    Q_inf_list =[get_docid(url), title, Q_detail, ID, Q_view, Q_date]
    A_inf_list =[]
    n = 1
    for i,j in zip(ans,A_dates):
        changed = i.replace(","," ")
        if j.endswith(" 전"):
            today = datetime.today().strftime('%Y.%m.%d.')
            A_inf_list.append([doc_ID, doc_ID+str(n), changed[2:],today])
        else:
            A_inf_list.append([doc_ID, doc_ID+str(n), changed[2:],j])
        n += 1
    Q_df.loc[Q_df.shape[0]] = Q_inf_list
    
    for i in A_inf_list:
        A_df.loc[A_df.shape[0]] = i
    return Q_df, A_df

# 질문 리스트에서 각 질문의 URL 값 가져오기

In [10]:
# 질문 목록에서 URL만 가져기기
urls = []
def get_url_from_list(search_url):
    global urls
    response = requests.get(search_url) 
    
    soup = BeautifulSoup(response.text, 'html.parser')

    elements = soup.find_all(class_=re.compile(r'^_nclicks:kls_new\.list'))
    
    for element in elements[1:]:
        href = "https://kin.naver.com"+element.get('href')
        urls.append(href)
    return urls

### list 페이지 설정
### 삭제 직전인 데이터는 못불러옴 예시로 보여주는 수 밖에 없다. 글로 설명 불가능

In [11]:
from datetime import datetime
current_date = datetime.now()
Today = current_date.strftime("%Y-%m-%d")

def collect_urls(region):
    try:
        for i in range(1,21):
            page = f"https://kin.naver.com/qna/list.naver?dirId={region}&queryTime={Today}%2017%3A33%3A26&page={i}"
            urls= get_url_from_list(page)
    except:
        pass
    return urls

# 11월 06일 기준 전체 지역 URL 크롤링(매핑테이블의 index를 사용) -> 현재 지역&플레이스에 나와있는 모든 질문 url을 수집
## 총 실행시간 약 1시간30분

In [12]:
print(local_lists)

[120115, 120116, 120117, 120118, 120119, 120120, 120121, 120122, 120123, 120124, 120125, 120126, 120127, 120128, 120129, 120130, 120131, 120132, 120133, 120134, 120135, 120136, 120137, 120138, 120139, 120203, 120204, 120205, 120206, 120207, 120208, 120209, 120210, 120211, 120212, 120213, 120214, 120215, 120216, 120217, 120218, 120306, 120307, 120308, 120309, 120310, 120311, 120312, 120313, 120404, 120405, 120406, 120407, 120408, 120409, 120410, 120411, 120412, 120413, 120504, 120505, 120506, 120507, 120508, 120603, 120604, 120605, 120606, 120607, 120701, 120702, 120703, 120704, 120705, 120802, 120803, 120804, 120805, 120806, 120807, 120808, 120809, 120810, 120811, 120812, 120813, 120814, 120815, 120816, 120817, 120818, 120819, 120905, 120906, 120907, 120908, 120909, 120910, 120911, 120912, 120913, 120914, 120915, 120916, 120917, 120918, 120919, 120920, 120921, 120922, 120923, 120924, 120925, 120926, 120927, 120928, 120929, 120930, 120931, 120932, 120933, 120934, 120935, 120936, 121001,

In [13]:
for region in tqdm(local_lists, desc="Processing URLs", ncols=100, leave=True):
    collect_urls(region)

Processing URLs: 100%|██████████████████████████████████████████| 257/257 [1:23:21<00:00, 19.46s/it]


In [14]:
len(urls)

12245

In [15]:
urls = [url for url in urls if not isinstance(url, list)]
len(urls)

12245

# 실제 데이터 불러오기(매핑 테이블 추후 SQL에서 join 예정)
## 45분 정도 소요

In [17]:
for i in tqdm(range(len(urls)), desc="Processing URLs", ncols=100, leave=True):
    url = urls[i]
    try:
        collect_information(url, Q_df, A_df)
    except:
        print(f"질문자가 질문을 삭제했습니다. - {url}")

Processing URLs:  19%|███████▉                                 | 2380/12245 [09:15<32:25,  5.07it/s]

질문자가 질문을 삭제했습니다. - https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12021607&docId=477934882


Processing URLs:  43%|█████████████████▍                       | 5218/12245 [19:48<27:31,  4.26it/s]

질문자가 질문을 삭제했습니다. - https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12091501&docId=478071091


Processing URLs: 100%|████████████████████████████████████████| 12245/12245 [45:46<00:00,  4.46it/s]


In [18]:
Q_df

,문서 번호,제목,질문,위치 고유번호,조회수,질문 날짜
0,478091663,질문대치동 독서실,밥 주는 관리형 독서실 추천 좀 해주세요. 여러개 알려주세요 러셀 시대 두각은 이미...,120115,8,2024.11.19
1,477933150,질문수험생 점빼는 곳,수험표 할인되고 점 잘빼는곳이 어디있을까요?? 서울도괜찮고 전라도 광주도 괜찮습니다...,120115,18,2024.11.19
2,478089640,질문대치세정 자리 선착순인가요??,이번에 대치 세정 처음 가는데 자리는 어떻게 앉는건지 궁금합니다.. 선착순으로 앉는...,120115,9,2024.11.19
3,478072408,질문디스코팡팡 어디가 사람 제일 많나요?,팡을 좀 다니고 싶은데 사람 옶는데는 너무 부담스러워서 못가겠고 사람 주말 내내 적...,120115,16,2024.11.19
4,478065243,질문강남 에스테틱 추천해주세요 ㅠ,강남에 에스테틱 마사지랑 피부 같이 받을 수 있는 곳으로 추천해주세요! 제가 일 끝...,120115,16,2024.11.19
...,...,...,...,...,...,...
17456,476537754,질문세종 CGV는 무인 주차장 인가요?,,121721,40,2024.10.14
17457,476260849,질문세종 몰리브 상가에 입점된 옷가게 브랜드는 뭐 뭐 있나...,세종 몰리브 상가에 입점된 옷가게 브랜드는 뭐 뭐 있나요,121721,34,2024.10.07
17458,475642460,질문종촌동 노래방,세종시 종촌동에 꿈끼카드 사용 가능한 노래방 있나요?,121721,19,2024.09.21
17459,478094629,질문세종시 아름동 사는 학생인데 세종여고 기숙사 갈수있나요,세종시 아름동에 살고있고 조치원에 있는 세종여고 기숙사에 들어가고 싶습니다. 성적은...,121723,7,2024.11.19


In [19]:
A_df

,문서 번호,답변 ID,답변,답변 날짜
0,478065243,4780652431,하세요서울 강남 지역에서 에스테틱과 마사지 피부 관리를 모두 받을 수 있는 곳을 ...,2024.11.19.
1,478057758,4780577581,이유는 영동대로와 영동대교가 이어졌기 때문 입니다2.또한 올림픽대로. 강변북로와...,2024.11.18.
2,478052397,4780523971,넷플릭스 시청 가능한것으로 알아요그래도 가는 곳에 넷플릭스 가능여부 확인하고 가세...,2024.11.18.
3,478052397,4780523972,룸카페. 영화룸. PC방 티빙 넥플릭스 설치된 곳 가능 합니다2.따라서 드라마...,2024.11.18.
4,478037606,4780376061,집 한정식집 추천합니다,2024.11.18.
...,...,...,...,...
18483,475803620,4758036201,마을 1단지 근처에는 다양한 맛집이 많이 있습니다. 아이들과 함께 가기 좋은 한식 ...,2024.09.25.
18484,475803620,4758036202,의꿈 아이들이 잘먹어서 가끔 가는편입니다.,2024.10.26.
18485,476391816,4763918161,에 신고를 해서 경찰이 학교랑 연락해서 찾는게 맞아요학교 입장에선 수사기관도 아니고...,2024.10.12.
18486,476537754,4765377541,CGV 무인 주차장 있습니다,2024.10.14.


In [20]:
Q_df.to_csv('Questions_new.csv', index = False)
A_df.to_csv('Answers_new.csv', index = False)

In [22]:
questions_new = pd.read_csv("Questions_new.csv")
questions_old = pd.read_csv("Questions_old.csv")

In [24]:
questions_old

,문서 번호,제목,질문,위치 고유번호,조회수,질문 날짜
0,477577274,질문강남 고터 지하상가,고터 지하상가 11월 14일 목요일 수능일때 휴무인가요?,120115,15,2024.11.07
1,477576108,질문압구정로데오역 물품보관함,현제 압구정로데오역에 물품보관함 있나요? 있으면 어느 위치에 있는건가요?,120115,15,2024.11.07
2,477573260,질문강남 110명가능한 회식장소 빔 프로젝터있는곳 추천 좀,안녕하세요. 12월에 회사 송년회를 할 예정인데 110명 정도 인원이 모일 장소를 ...,120115,13,2024.11.07
3,477570620,질문개포중학교 질문,개포중에 잘생긴사람 많나요..? 많진 않아도 있긴 한가요..?ㅠ,120115,19,2024.11.07
4,477561630,질문가격문의요,제주에서 강남까지 박스한개 가격과 소요시간문의요 접수는 언제까지 하는지도요,120115,14,2024.11.07
...,...,...,...,...,...,...
12285,476391816,질문번개장터 사기학생 학교 아는데 학교에 전화해도,제목 그대로 번개장터에서 사기를 당했는데 사기학생이 11년생 어디학교 몇반인지 알게...,121719,27,2024.10.09
12286,476537754,질문세종 CGV는 무인 주차장 인가요?,NaN,121721,27,2024.10.14
12287,476260849,질문세종 몰리브 상가에 입점된 옷가게 브랜드는 뭐 뭐 있나...,세종 몰리브 상가에 입점된 옷가게 브랜드는 뭐 뭐 있나요,121721,30,2024.10.07
12288,475642460,질문종촌동 노래방,세종시 종촌동에 꿈끼카드 사용 가능한 노래방 있나요?,121721,17,2024.09.21


In [26]:
combined = pd.concat([questions_new, questions_old]).drop_duplicates(subset='문서 번호', keep='first')
print(combined)

           문서 번호                             제목  \
0      478091663                      질문대치동 독서실   
1      477933150                    질문수험생 점빼는 곳   
2      478089640             질문대치세정 자리 선착순인가요??   
3      478072408         질문디스코팡팡 어디가 사람 제일 많나요?   
4      478065243             질문강남 에스테틱 추천해주세요 ㅠ   
...          ...                            ...   
12280  475561333            질문빨리 빨리 좀 답변 보내 주시오   
12281  475505530                   질문제발 부탁해요 진심   
12282  475169557  질문동대구 푸르지오 브리센트 수성구 학군 확실한가요?   
12283  475373491                    질문조치원역 와이파이   
12284  476649506  질문나성동 관산성한우 세종 나성점 맛집 콜키지되나요?   

                                                      질문  위치 고유번호  조회수  \
0      밥 주는 관리형 독서실 추천 좀 해주세요. 여러개 알려주세요 러셀 시대 두각은 이미...   120115    8   
1      수험표 할인되고 점 잘빼는곳이 어디있을까요?? 서울도괜찮고 전라도 광주도 괜찮습니다...   120115   18   
2      이번에 대치 세정 처음 가는데 자리는 어떻게 앉는건지 궁금합니다.. 선착순으로 앉는...   120115    9   
3      팡을 좀 다니고 싶은데 사람 옶는데는 너무 부담스러워서 못가겠고 사람 주말 내내 적...   120115   16   
4      강남에 에스테틱 마